We consider a term of the form $u\text{cos}(2\beta\phi)$ to see if it is relevant in the IR. We start by generating samples, specifying the RG scheme, and then re learning after coarse-graining.

In [ ]:
using AdvancedHMC, ForwardDiff
using LogDensityProblems, LinearAlgebra, Random, Statistics, Optim, GalacticOptim, Plots

# 2D Sine-Gordon Model Definition
struct SineGordonModel2D
    lattice_size::Int   # Lattice size (NxN)
    α::Float64          # Coupling strength for the sine term
    β::Float64          # Mass parameter (m)
    u :: Float64        #coeff of cos 2betaphi
end

# Total energy (kinetic + gradient + sine potential)
function calculate_energy_2d(phi::Matrix{T}, model::SineGordonModel2D) where T<:Real
    N = size(phi,1)
    kin = zero(T)
    grad = zero(T)
    sine = zero(T)
    α, β, u = model.α, model.β, model.u

    for i in 1:N, j in 1:N
        ip = mod1(i+1, N); im = mod1(i-1, N)
        jp = mod1(j+1, N); jm = mod1(j-1, N)
        # Kinetic term: discrete gradient
        kin  += 0.5 * ((phi[i,j] - phi[ip,j])^2 + (phi[i,j] - phi[i,jp])^2)
        # Sine interaction potential
        sine += (α/β^2) * (1 - cos(β * phi[i,j])) + u * cos(2*β*phi[i,j])
    end
    return kin + sine
end

# HMC log-density wrapper
struct SGTarget2D
    model::SineGordonModel2D
end

function LogDensityProblems.logdensity(p::SGTarget2D, x::Vector{T}) where T<:Real
    N = p.model.lattice_size
    phi = reshape(x, N, N)
    return -calculate_energy_2d(phi, p.model)
end

LogDensityProblems.dimension(p::SGTarget2D) = p.model.lattice_size^2
LogDensityProblems.capabilities(::Type{SGTarget2D}) = LogDensityProblems.LogDensityOrder{1}()

# HMC data generation function
function generate_sg_data_hmc_2d(model::SineGordonModel2D, n_samples::Int; n_adapts::Int=2000)
    N = model.lattice_size
    target     = SGTarget2D(model)
    init_field = randn(N, N)

    metric     = DiagEuclideanMetric(N^2)
    ham        = Hamiltonian(metric, target, ForwardDiff)
    ε          = find_good_stepsize(ham, vec(init_field))
    integrator = Leapfrog(ε)
    kernel     = HMCKernel(Trajectory{MultinomialTS}(integrator, GeneralisedNoUTurn()))
    adaptor    = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator))

    samples_vec, _ = sample(ham, kernel, vec(init_field), n_samples, adaptor, n_adapts; progress=false)
    return [reshape(s, N, N) for s in samples_vec]
end

# Score matching objective (no spacing parameter)
function score_matching_objective_2d(α::Float64, β::Float64, u::Float64, data::Vector{Matrix{Float64}})
    N = size(data[1],1)
    total_obj = 0.0
    for φ in data
        for i in 1:N, j in 1:N
            x = φ[i,j]
            # Periodic neighbors
            xL = φ[mod1(i-1,N), j]; xR = φ[mod1(i+1,N), j]
            xD = φ[i, mod1(j-1,N)]; xU = φ[i, mod1(j+1,N)]
            lap = 4*x - (xL + xR + xD + xU)

            # Score and its derivative
            score = -(lap + (α/β) * sin(β * x) - 2*u*β*sin(2*β*x))
            ds    = -(4 + α * cos(β * x) - 4 * u * β^2 * cos(2 * β * x))

            total_obj += 0.5 * score^2 + ds
        end
    end
    return total_obj / length(data)
end

# Estimate α, β by Particle Swarm optimization
function estimate_parameters_score_matching_2d(data::Vector{Matrix{Float64}})
    objective(x) = score_matching_objective_2d(x[1], x[2], x[3],  data)
    initial = [0.1, 1.0, 0.5]
    lower   = [0.01, 0.01, 0.01]; upper = [10.0, 10.0, 10.0]
    ps      = ParticleSwarm(lower = lower, upper = upper)
    result  = optimize(objective, initial, ps)
    return result.minimizer[1], result.minimizer[2], result.minimizer[3]
end

# Compute raw gradient squared measure
function compute_raw_grad_squared(phi::Matrix{Float64})
    gradx = circshift(phi, (-1, 0)) .- phi
    grady = circshift(phi, (0, -1)) .- phi
    return mean(gradx.^2 .+ grady.^2)
end

# Block-spin coarse-graining with kinetic renormalization
function block_spin_transform(phi::Matrix{Float64})
    nx, ny = size(phi)
    @assert nx % 2 == 0 && ny % 2 == 0
    nx2, ny2 = nx÷2, ny÷2
    return [ median(vec(phi[2i-1:2i, 2j-1:2j])) 
             for i in 1:nx2, j in 1:ny2 ]
end

block_spin_samples(data::Vector{Matrix{Float64}}) = [ block_spin_transform(φ) for φ in data ]
# RG flow loop
function run_rg_procedure(model::SineGordonModel2D, n_samples::Int, n_steps::Int, reps::Int)
    alphas = Float64[]; betas = Float64[]; us = Float64[]; sizes = Int[]
    push!(alphas, model.α); push!(betas, model.β); push!(us, model.u); push!(sizes, model.lattice_size)

    samples = generate_sg_data_hmc_2d(model, n_samples)
    for step in 1:n_steps
        samples = block_spin_samples(samples)
        new_size = size(samples[1],1)

        αs = Float64[]; βs = Float64[]; vs = Float64[]
        for rep in 1:reps
            α̂, β̂,u = estimate_parameters_score_matching_2d(samples)
            push!(αs, α̂); push!(βs, β̂); push!(vs,u)
        end

        push!(alphas, mean(αs)); push!(betas, mean(βs)); push!(us, mean(vs)); push!(sizes, new_size)
    end
    return alphas, betas, us, sizes
end

# Example usage
model0 = SineGordonModel2D(32, 3.0, sqrt(8π), 1.5)
println("\nVerifying parameter recovery on original lattice:")
samples0 = generate_sg_data_hmc_2d(model0, 10000)
α_est0, β_est0, u_est0 = estimate_parameters_score_matching_2d(samples0)
println("True α = ", model0.α, ", Estimated α = ", α_est0)
println("True β = ", model0.β, ", Estimated β = ", β_est0)
println("True u = ", model0.u, ", Estimated u = ", u_est0)

alphas, betas, us, sizes = run_rg_procedure(model0, 10000, 3, 2)

p = plot(sizes, alphas; label="α", marker=:o, lw=2,
         xlabel="Lattice size", ylabel="Parameter", xscale=:log2)
plot!(sizes, betas; label="β", marker=:s, lw=2)
plot!(sizes, us; label = "u", marker = :square, lw = 2)
display(p)
savefig(p, "sg_rg_flow_cos2phi.png")


As usual, we perform RG separately, where parameters after RG are named as {parameter}_{RG step}.


In [ ]:
samples1 = block_spin_samples(samples0)
α_est1, β_est1, u_est1 = estimate_parameters_score_matching_2d(samples1)

In [ ]:
samples2 = block_spin_samples(samples1)
α_est2, β_est2, u_est2 = estimate_parameters_score_matching_2d(samples2)

In [ ]:
samples3 = block_spin_samples(samples2)
α_est3, β_est3, u_est3 = estimate_parameters_score_matching_2d(samples3)